# HW4 :  ETF 評比績效理論實作 

In [1]:
import pandas as pd
import numpy as np
import datetime as dt
from scipy.optimize import fsolve
from scipy import stats
import math
from sklearn.neighbors.kde import KernelDensity
from sympy import *
from scipy.stats.kde import gaussian_kde
from scipy.stats import norm
from numpy import linspace, hstack
import os
import time

In [2]:
def get_data_from_csv(file):
    df = []
    if os.path.isfile(file):
        df = pd.read_csv(file,index_col=1)
        df = df.drop(['sn'], axis=1)
    return df

def cal_sharp_ratio(data):
    total = len(data)
    data = np.log(data)
    mean = (data.mean())
    std = data.std()
    skew = data.skew()
    kurt = data.kurt()
    var = -1.96
    Z = var + (skew * ((var**2) - 1) / 6) + (kurt * ((var**3) - 3*var) / 24) - ((skew**2) * (2*(var**3) - 5*var) / 36)
    adj_var = (-0.5)*(std**2)*52-Z*std*math.sqrt(total)
    cumprod_return = np.exp(data).cumprod()[-1]
    sharpe_ratio = cumprod_return / adj_var
    return sharpe_ratio

def cal_omega(data,L):
    probDensityFun = gaussian_kde(data)
    x = np.linspace(-0.5,0.5,10000)        #將數據用平滑的方式算單點機率
    p = (probDensityFun(x) / probDensityFun(x).sum())
    p = p.cumsum()
    p = pd.DataFrame(p,index=x)
    omega = ((1-p).loc[L/100/52:].sum()) / (p.loc[:L/100/52].sum())
    return omega[0]

def f1(risk,returns):
    return sum( np.exp(-1*returns/risk) ) - len(returns)

def cal_riskiness_r(data):
    rfrate = 2.45785 / 12 / 100  #月報酬
    data = data-1 - rfrate
    guess = 10**(-5)
    while (guess<1000):
        risk = fsolve(f1,guess,data)
        if risk[0] != guess:
            break
        guess = guess*10
    return np.exp(-1/risk[0])

# 讀取ETF原始資料

In [3]:
# USG3M.xlsx檔案裡有三組資料，但因為沒有head，看不懂是什麼資料，似乎與無風險利率有關
bond_rate = pd.read_excel("data/USG3M.xlsx",index_col=0,parse_dates=True,header=None)

# 取第一組資料
USG3M = bond_rate[1]

# 取得今天日期 (月/日/年)
#today = str(dt.datetime.today().month)+"/"+str(dt.datetime.today().day)+"/"+str(dt.datetime.today().year)

# 取出ETF資料
etf_data_weekly = get_data_from_csv('./data/weekly/Emerging Markets ETF List (79)_merge.csv')
etf_data_weekly = etf_data_weekly.dropna()
print('週資料共',etf_data_weekly.shape[0],'筆')

etf_data_monthly = get_data_from_csv('./data/monthly/Emerging Markets ETF List (79)_merge.csv')
etf_data_monthly = etf_data_monthly.dropna()
print('月資料共',etf_data_monthly.shape[0],'筆')

# 將ETF price換算成return
etf_return_weekly = etf_data_weekly / etf_data_weekly.shift(1)
etf_return_monthly = etf_data_monthly / etf_data_monthly.shift(1)
print(etf_return_weekly.shape)
print(etf_return_monthly.shape)

# 移除無限大的資料
etf_return_weekly = etf_return_weekly.replace([np.inf, -np.inf], np.nan).dropna()
etf_return_monthly = etf_return_monthly.replace([np.inf, -np.inf], np.nan).dropna()
print(etf_return_weekly.shape)
print(etf_return_monthly.shape)

# 取出ETF名稱(簡寫)
etf_symbols = list(etf_data_weekly.columns)
print(etf_symbols)

週資料共 173 筆
月資料共 40 筆
(173, 48)
(40, 44)
(172, 48)
(39, 44)
['ADRE', 'BBRC', 'BICK', 'BKF', 'DBEM', 'DEM', 'DGRE', 'DGS', 'DVYE', 'EDC', 'EDIV', 'EDOG', 'EDZ', 'EEB', 'EELV', 'EEM', 'EEMO', 'EEMS', 'EET', 'EEV', 'EMCG', 'EMFM', 'EMIF', 'EMQQ', 'EUM', 'EWEM', 'EWX', 'FEM', 'FEMS', 'FNDE', 'GEM', 'HEEM', 'HILO', 'IEMG', 'JPEM', 'PIE', 'PXH', 'PXR', 'QEMM', 'ROAM', 'SCHE', 'SDEM', 'SPEM', 'TLEH', 'TLTE', 'VWO', 'XCEM', 'XSOE']


In [4]:
etf_return_weekly.shape[1]

48

# 指標A : Adjust Sharpe Ratios

### sharpe ratio 週資料

In [5]:
sharpe_ratio_weekly = {}

# 取出每支ETF的return，計算sharpe ratio
for i in range(etf_return_weekly.shape[1]):
    etf_name = etf_symbols[i]
    etf_return = etf_return_weekly.iloc[:,i]
    sharp_ratio = cal_sharp_ratio(etf_return)
    sharpe_ratio_weekly[etf_name] = sharp_ratio
    
# 排序，取前10名
sorted_sharpe_ratio_weekly = sorted(sharpe_ratio_weekly.items(), key=lambda kv: kv[1])
top_sharpe_ratio_weekly = sorted_sharpe_ratio_weekly[:10]

# 整理dataframe
ranking_sharpe_ratio_weekly = pd.DataFrame(top_sharpe_ratio_weekly,columns=['ETF_Symbol','val'])
ranking_sharpe_ratio_weekly['Ranking'] = list(range(1,11))
ranking_sharpe_ratio_weekly = ranking_sharpe_ratio_weekly.drop(['val'],axis=1)
ranking_sharpe_ratio_weekly = ranking_sharpe_ratio_weekly.set_index('Ranking')

print('done')

done


### sharpe ratio 月資料

In [6]:
sharpe_ratio_monthly = {}

# 取出每支ETF的return，計算sharpe ratio
for i in range(etf_return_monthly.shape[1]):
    etf_name = etf_symbols[i]
    etf_return = etf_return_monthly.iloc[:,i]
    sharp_ratio = cal_sharp_ratio(etf_return)
    sharpe_ratio_monthly[etf_name] = sharp_ratio
    
# 排序，取前10名
sorted_sharpe_ratio_monthly = sorted(sharpe_ratio_monthly.items(), key=lambda kv: kv[1])
top_sharpe_ratio_monthly = sorted_sharpe_ratio_monthly[:10]
ranking_sharpe_ratio_monthly = pd.DataFrame(top_sharpe_ratio_monthly,columns=['ETF_Symbol','val'])

# 整理dataframe
ranking_sharpe_ratio_monthly = pd.DataFrame(top_sharpe_ratio_monthly,columns=['ETF_Symbol','val'])
ranking_sharpe_ratio_monthly['Ranking'] = list(range(1,11))
ranking_sharpe_ratio_monthly = ranking_sharpe_ratio_monthly.drop(['val'],axis=1)
ranking_sharpe_ratio_monthly = ranking_sharpe_ratio_monthly.set_index('Ranking')

print('done')

done


# 指標B : Omega 

### omega 週資料

In [7]:
omega_weekly = {}

# 取出每支ETF的return資料計算omega
for i in range(etf_return_weekly.shape[1]):
    etf_name = etf_symbols[i]
    etf_return = etf_return_weekly.iloc[:,i]
    L = USG3M.loc[etf_return.index[-1]]
    omega = cal_omega(etf_return,L)
    # 應移除NaN ???
    if not math.isnan(omega):
        omega_weekly[etf_name] = omega

# 排序, 取前10名
sorted_omega_weekly = sorted(omega_weekly.items(), key=lambda kv: kv[1])
omega_top_weekly = sorted_omega_weekly[:10]

# 整理dataframe
ranking_omega_weekly = pd.DataFrame(omega_top_weekly,columns=['ETF_Symbol','val'])
ranking_omega_weekly['Ranking'] = list(range(1,ranking_omega_weekly.shape[0]+1))
ranking_omega_weekly = ranking_omega_weekly.drop(['val'],axis=1)
ranking_omega_weekly = ranking_omega_weekly.set_index('Ranking')

print('done')

C:\Users\R\Anaconda3\lib\site-packages\ipykernel_launcher.py:25: RuntimeWarning: invalid value encountered in true_divide


done


### omega 月資料

In [8]:
omega_monthly = {}

# 取出每支ETF的return資料計算omega
for i in range(etf_return_monthly.shape[1]):
    #print(i)
    etf_name = etf_symbols[i]
    etf_return = etf_return_monthly.iloc[:,i]
    L = USG3M.loc[etf_return.index[-1]]
    omega = cal_omega(etf_return,L)
    #print(omega)
    # 應移除NaN ???
    if not math.isnan(omega):
        omega_monthly[etf_name] = omega

# 排序, 取前10名
sorted_omega_monthly = sorted(omega_monthly.items(), key=lambda kv: kv[1])
omega_top_monthly = sorted_omega_monthly[:10]

# 整理dataframe
ranking_omega_monthly = pd.DataFrame(omega_top_monthly,columns=['ETF_Symbol','val'])
ranking_omega_monthly['Ranking'] = list(range(1,ranking_omega_monthly.shape[0]+1))
ranking_omega_monthly = ranking_omega_monthly.drop(['val'],axis=1)
ranking_omega_monthly = ranking_omega_monthly.set_index('Ranking')

print('done')

done


# 指標C : Riskiness R

### riskiness r 週資料

In [9]:
riskiness_r_weekly = {}

for i in range(etf_return_weekly.shape[1]):
    etf_name = etf_symbols[i]
    etf_return = etf_return_weekly.iloc[:,i]
    r = cal_riskiness_r(etf_return)
    riskiness_r_weekly[etf_name] = r

# 排序, 取前10名
sorted_riskiness_r_weekly = sorted(riskiness_r_weekly.items(), key=lambda kv: kv[1])
top_riskiness_r_weekly = sorted_riskiness_r_weekly[:10]

# 整理dataframe
ranking_riskiness_r_weekly = pd.DataFrame(top_riskiness_r_weekly,columns=['ETF_Symbol','val'])
ranking_riskiness_r_weekly['Ranking'] = list(range(1,11))
ranking_riskiness_r_weekly = ranking_riskiness_r_weekly.drop(['val'],axis=1)
ranking_riskiness_r_weekly = ranking_riskiness_r_weekly.set_index('Ranking')

print('done')

C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


done


### riskiness r 月資料

In [10]:
riskiness_r_monthly = {}

for i in range(etf_return_monthly.shape[1]):
    etf_name = etf_symbols[i]
    etf_return = etf_return_monthly.iloc[:,i]
    r = cal_riskiness_r(etf_return)
    riskiness_r_monthly[etf_name] = r

# 排序, 取前10名
sorted_riskiness_r_monthly = sorted(riskiness_r_monthly.items(), key=lambda kv: kv[1])
top_riskiness_r_monthly = sorted_riskiness_r_monthly[:10]

# 整理dataframe
ranking_riskiness_r_monthly = pd.DataFrame(top_riskiness_r_monthly,columns=['ETF_Symbol','val'])
ranking_riskiness_r_monthly['Ranking'] = list(range(1,11))
ranking_riskiness_r_monthly = ranking_riskiness_r_monthly.drop(['val'],axis=1)
ranking_riskiness_r_monthly = ranking_riskiness_r_monthly.set_index('Ranking')

print('done')

C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The iteration is not making good progress, as measured by the 
  improvement from the last ten iterations.
  warnings.warn(msg, RuntimeWarning)
C:\Users\R\Anaconda3\lib\site-packages\scipy\optimize\minpack.py:163: RuntimeWarning: The number of calls to function has reached maxfev = 400.
  warnings.warn(msg, RuntimeWarning)


done


# 週資料排名

In [11]:
print('週資料分析組')
print('==========')

print('A 評比指標')
print('----------')
print(ranking_sharpe_ratio_weekly)
print()

print('B 評比指標')
print('----------')
print(ranking_omega_weekly)
print()

print('C 評比指標')
print('----------')
print(ranking_riskiness_r_weekly)
print()

週資料分析組
A 評比指標
----------
        ETF_Symbol
Ranking           
1              EDZ
2              PXR
3              EEV
4              EDC
5              EUM
6              EET
7             EEMO
8             EMQQ
9              PIE
10            EMCG

B 評比指標
----------
        ETF_Symbol
Ranking           
1              PXR
2              EDZ
3              EDC
4             EEMO
5              EET
6              EEV
7             EMQQ

C 評比指標
----------
        ETF_Symbol
Ranking           
1              EWX
2             HEEM
3             HILO
4             QEMM
5             ROAM
6             XCEM
7              PXH
8             FNDE
9              EEB
10            DVYE



# 月資料排名

In [12]:
print('月資料分析組')
print('==========')

print('A 評比指標')
print('----------')
print(ranking_sharpe_ratio_monthly)
print()

print('B 評比指標')
print('----------')
print(ranking_omega_monthly)
print()

print('C 評比指標')
print('----------')
print(ranking_riskiness_r_monthly)
print()

月資料分析組
A 評比指標
----------
        ETF_Symbol
Ranking           
1              EDZ
2              EEV
3             EMQQ
4             EEMO
5              EDC
6              EET
7             EMIF
8              PIE
9             EEMS
10            IEMG

B 評比指標
----------
        ETF_Symbol
Ranking           
1              EDZ
2              EDC
3              EET
4              EEV
5             EMIF
6             EEMO
7              PIE
8             FEMS
9             EDOG
10             FEM

C 評比指標
----------
        ETF_Symbol
Ranking           
1             ADRE
2             BICK
3             DVYE
4             EELV
5             SCHE
6             JPEM
7             FEMS
8             HILO
9              DGS
10             EUM



# 合併排列

### 週資料

In [13]:
c1 = ranking_sharpe_ratio_weekly['ETF_Symbol'].values.tolist()

c2 = ranking_omega_weekly['ETF_Symbol'].values.tolist()
# 補0
for i in range(10):
    if i>=len(c2):
        c2.append('-')
        
c3 = ranking_riskiness_r_weekly['ETF_Symbol'].values.tolist()

dict1 = {'Ranking':ranking_sharpe_ratio_weekly.index,'指標A':c1,'指標B':c2,'指標C':c3}
weekly_reanking = pd.DataFrame(dict1)

print('週資料')
print(weekly_reanking)
print()

週資料
   Ranking   指標A   指標B   指標C
0        1   EDZ   PXR   EWX
1        2   PXR   EDZ  HEEM
2        3   EEV   EDC  HILO
3        4   EDC  EEMO  QEMM
4        5   EUM   EET  ROAM
5        6   EET   EEV  XCEM
6        7  EEMO  EMQQ   PXH
7        8  EMQQ     -  FNDE
8        9   PIE     -   EEB
9       10  EMCG     -  DVYE



### 月資料

In [14]:
c1 = ranking_sharpe_ratio_monthly['ETF_Symbol'].values.tolist()

c2 = ranking_omega_monthly['ETF_Symbol'].values.tolist()
# 補0
for i in range(10):
    if i>=len(c2):
        c2.append('-')
        
c3 = ranking_riskiness_r_monthly['ETF_Symbol'].values.tolist()

dict1 = {'Ranking':ranking_sharpe_ratio_monthly.index,'指標A':c1,'指標B':c2,'指標C':c3}
monthly_reanking = pd.DataFrame(dict1)

print('月資料')
print(monthly_reanking)
print()

月資料
   Ranking   指標A   指標B   指標C
0        1   EDZ   EDZ  ADRE
1        2   EEV   EDC  BICK
2        3  EMQQ   EET  DVYE
3        4  EEMO   EEV  EELV
4        5   EDC  EMIF  SCHE
5        6   EET  EEMO  JPEM
6        7  EMIF   PIE  FEMS
7        8   PIE  FEMS  HILO
8        9  EEMS  EDOG   DGS
9       10  IEMG   FEM   EUM

